In [94]:
import pandas as pd

file_url = 'C:\\Users\\imaja\\OneDrive\\Desktop\\MBA 2022 -2024\\T4\\Coding Competition\\d01d13b9-fe5a-429c-8b1a-7e2259648102easetripTrain.csv'

train_df = pd.read_csv(file_url)

In [95]:
with pd.option_context("display.max_columns", None, "display.max_rows", None):
    display(train_df.head())

,transaction_id,date,time,train_number,start_port_id,start_port_metro_or_not,end_port_id,end_port_metro_or_not,holiday_or_not,zone,distance,class,current_status,seating_capacity,divyang_or_not,medical_or_not,number_of_ports_in_between,Price
0,0,11-09-1997,6-21-42,NaN,2561,2,3210,1,1,NE,3905,6,56,471.0,2,2,30,3084
1,1,02-10-1998,5-20-51,24200.0,1419,2,8405,2,2,SE,2314,4,18,229.0,2,1,20,1735
2,2,21-03-1993,12-57-26,24250.0,6646,1,3071,1,1,S,3411,4,46,247.0,1,1,9,613
3,3,06-Oct-02,20-29-10,32289.0,7339,2,1742,1,2,E,3403,4,15,196.0,2,2,16,3164
4,4,19-08-2009,22-41-2,27635.0,5320,2,458,2,1,W,2829,2,17,123.0,2,1,14,2574


In [96]:
train_df.shape

(80000, 18)

In [97]:
train_df.isnull().sum()

transaction_id                0
date                          0
time                          0
train_number                  1
start_port_id                 0
start_port_metro_or_not       0
end_port_id                   0
end_port_metro_or_not         0
holiday_or_not                0
zone                          0
distance                      0
class                         0
current_status                0
seating_capacity              1
divyang_or_not                0
medical_or_not                0
number_of_ports_in_between    0
Price                         0
dtype: int64

In [98]:
train_df.dropna(inplace=True)

In [99]:
train_df.isnull().sum()

transaction_id                0
date                          0
time                          0
train_number                  0
start_port_id                 0
start_port_metro_or_not       0
end_port_id                   0
end_port_metro_or_not         0
holiday_or_not                0
zone                          0
distance                      0
class                         0
current_status                0
seating_capacity              0
divyang_or_not                0
medical_or_not                0
number_of_ports_in_between    0
Price                         0
dtype: int64

In [100]:
train_df.shape

(79998, 18)

# ML

In [101]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [102]:
X = train_df.drop(['Price', 'date', 'time','transaction_id','divyang_or_not','class','zone'], axis=1)  # Independent variables
y = train_df['Price']  # Dependent variable

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [104]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [105]:
# Make predictions using the test set
y_pred = model.predict(X_test)

In [106]:
import numpy as np

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 705.4095088402429


# Model Report

In [38]:
import statsmodels.api as sm

# Add a constant term (intercept) to the independent variables
X = sm.add_constant(X)

# Fit a linear regression model using statsmodels
model1 = sm.OLS(y, X).fit()

# Print a summary report of the regression model
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.421
Model:                            OLS   Adj. R-squared:                  0.421
Method:                 Least Squares   F-statistic:                     2648.
Date:                Sun, 08 Oct 2023   Prob (F-statistic):               0.00
Time:                        01:46:40   Log-Likelihood:            -6.3807e+05
No. Observations:               79998   AIC:                         1.276e+06
Df Residuals:                   79975   BIC:                         1.276e+06
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

# Stepwise Regression Report

In [40]:
import pandas as pd
import statsmodels.api as sm

# Assuming 'train_df' contains your dataset with 'zone' one-hot encoded
# Separate the dependent variable 'Price' and independent variables
X = train_df.drop(['Price', 'date', 'time'], axis=1)  # Independent variables
y = train_df['Price']  # Dependent variable

# Perform forward stepwise regression
def forward_stepwise_selection(X, y, initial_list=[], threshold_in=0.01, verbose=True):
    included = list(initial_list)
    while True:
        changed = False
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=float)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            if verbose:
                print(f'Add {best_feature:30} with p-value {best_pval}')
            changed = True
        if not changed:
            break
    return included

# Perform forward stepwise selection
selected_features = forward_stepwise_selection(X, y)

# Create a new DataFrame with the selected features
X_selected = X[selected_features]

# Fit a linear regression model using the selected features
model = sm.OLS(y, sm.add_constant(X_selected)).fit()

# Print a summary report of the regression model
print(model.summary())

Add distance                       with p-value 0.0
                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.421
Model:                            OLS   Adj. R-squared:                  0.421
Method:                 Least Squares   F-statistic:                 5.824e+04
Date:                Sun, 08 Oct 2023   Prob (F-statistic):               0.00
Time:                        01:51:23   Log-Likelihood:            -6.3808e+05
No. Observations:               79998   AIC:                         1.276e+06
Df Residuals:                   79996   BIC:                         1.276e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------


# Predicting for new transaction IDs

In [88]:
import pandas as pd

file_url = 'C:\\Users\\imaja\\OneDrive\\Desktop\\MBA 2022 -2024\\T4\\Coding Competition\\Test.csv'

test_df = pd.read_csv(file_url)

In [89]:
test_df

,transaction_id,date,time,train_number,start_port_id,start_port_metro_or_not,end_port_id,end_port_metro_or_not,holiday_or_not,zone,distance,class,current_status,seating_capacity,divyang_or_not,medical_or_not,number_of_ports_in_between
0,80000,Dec-97,10-56-42,21379,5176,1,138,1,1,W,3490,5,51,598,2,1,24
1,80001,09-Oct-00,19-29-6,13392,4435,2,58,1,2,SE,4055,5,24,474,1,1,11
2,80002,30-01-1992,5-29-5,26952,2812,2,6215,2,1,C,657,3,25,347,2,2,27
3,80003,08-09-2016,6-43-4,14035,3613,1,4996,2,2,NEF,1776,2,27,226,1,1,8
4,80004,06-05-2008,9-39-28,26962,3180,1,5969,2,1,NW,4049,1,23,135,1,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19996,99996,14-02-2002,20-26-35,28162,1637,2,6407,1,1,NE,494,6,82,598,1,2,12
19997,99997,11-Aug-91,20-40-11,12576,875,1,1223,2,1,NW,1221,5,57,319,1,2,28
19998,99998,16-08-1993,15-23-54,33011,4487,1,832,1,1,NE,1151,1,17,205,1,2,3
19999,99999,09-03-2008,23-48-19,22272,1419,1,4100,1,2,NE,1412,3,13,262,1,2,21


In [91]:
test_df.isnull().sum()

transaction_id                0
date                          0
time                          0
train_number                  0
start_port_id                 0
start_port_metro_or_not       0
end_port_id                   0
end_port_metro_or_not         0
holiday_or_not                0
zone                          0
distance                      0
class                         0
current_status                0
seating_capacity              0
divyang_or_not                0
medical_or_not                0
number_of_ports_in_between    0
dtype: int64

In [110]:
test_df['Price'] = model.predict(test_df.drop(['date', 'time', 'transaction_id', 'divyang_or_not', 'class', 'zone'], axis=1))

In [111]:
test_df

,transaction_id,date,time,train_number,start_port_id,start_port_metro_or_not,end_port_id,end_port_metro_or_not,holiday_or_not,zone,distance,class,current_status,seating_capacity,divyang_or_not,medical_or_not,number_of_ports_in_between,Price
0,80000,Dec-97,10-56-42,21379,5176,1,138,1,1,W,3490,5,51,598,2,1,24,1703.988938
1,80001,09-Oct-00,19-29-6,13392,4435,2,58,1,2,SE,4055,5,24,474,1,1,11,1985.876984
2,80002,30-01-1992,5-29-5,26952,2812,2,6215,2,1,C,657,3,25,347,2,2,27,339.020887
3,80003,08-09-2016,6-43-4,14035,3613,1,4996,2,2,NEF,1776,2,27,226,1,1,8,882.347384
4,80004,06-05-2008,9-39-28,26962,3180,1,5969,2,1,NW,4049,1,23,135,1,2,8,2013.505268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19996,99996,14-02-2002,20-26-35,28162,1637,2,6407,1,1,NE,494,6,82,598,1,2,12,251.748679
19997,99997,11-Aug-91,20-40-11,12576,875,1,1223,2,1,NW,1221,5,57,319,1,2,28,617.243453
19998,99998,16-08-1993,15-23-54,33011,4487,1,832,1,1,NE,1151,1,17,205,1,2,3,560.699267
19999,99999,09-03-2008,23-48-19,22272,1419,1,4100,1,2,NE,1412,3,13,262,1,2,21,695.739814


In [112]:
test_df.isnull().sum()

transaction_id                0
date                          0
time                          0
train_number                  0
start_port_id                 0
start_port_metro_or_not       0
end_port_id                   0
end_port_metro_or_not         0
holiday_or_not                0
zone                          0
distance                      0
class                         0
current_status                0
seating_capacity              0
divyang_or_not                0
medical_or_not                0
number_of_ports_in_between    0
Price                         0
dtype: int64

# Export CSV

In [115]:
file_url = 'C:\\Users\\imaja\\OneDrive\\Desktop\\MBA 2022 -2024\\T4\\Coding Competition\\Test2.csv'


test_df.to_csv(file_url, index=False)